# ChatBoat

##### we are go over an example of how to design and implement an LLM - powered chatbot.This chatbot will be able to have a conversation and remember previous interactions.
##### note this chatbot that will only use the language model to have a conversation.there are several other related concept that you may be looking for:

##### * conversation RAG:enable a chatbot experience over an external source of data

##### * Agent: Build a chatBot that can take actions

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

groq_api_key=os.getenv("groq_api_key")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq

model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

c:\Users\siddh\anaconda3\envs\langenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020049C35870>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020049C35780>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="hi,i am siddhesh and iam aspirant data analyst")])

AIMessage(content="Nice to meet you, Siddhesh! As a data analyst aspirant, you're taking the first step towards a fascinating career path. Data analysis is a crucial aspect of decision-making in various industries, and the demand for data analysts is on the rise.\n\nWhat specific areas of data analysis are you interested in, or what do you want to achieve as a data analyst? Are you looking to:\n\n1. Work in a specific industry (e.g., finance, healthcare, marketing)?\n2. Gain a strong foundation in statistical modeling and machine learning?\n3. Develop expertise in data visualization and storytelling?\n4. Explore tools like Excel, SQL, Python, R, or Tableau?\n5. Something else?\n\nFeel free to share your goals, and I'll do my best to provide guidance and resources to help you on your journey!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 168, 'prompt_tokens': 48, 'total_tokens': 216, 'completion_time': 0.242718391, 'prompt_time': 0.002761615, 'queue_tim

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="hi,i am siddhesh and iam aspirant data analyst"),
        AIMessage(content="hello siddhesh ! it's nice to meet you. \n\n as a data anslyst"),
        HumanMessage(content="hey What's my name and what do I do?")
    ]
)

AIMessage(content='Your name is Siddhesh, and you are an aspirant Data Analyst.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 89, 'total_tokens': 106, 'completion_time': 0.033081017, 'prompt_time': 0.007876673, 'queue_time': 0.048513947, 'total_time': 0.04095769}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--735aac5c-8a08-4339-921f-ad631c1f900a-0', usage_metadata={'input_tokens': 89, 'output_tokens': 17, 'total_tokens': 106})

### Message History
we can use a message history class to wrap our model and make it startful. this will keep track of input and output of the model and store them in some datastore. Future interactions will then load those messages and pss them into the chain as part of the input .let's see how to use this!

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}


def get_session_history(session_id:str)->BaseChatMessageHistory: # to save session of each and every use seperate 
    if session_id not in store:
          store[session_id]=ChatMessageHistory()

    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [13]:
config={"configurable":{"session_id":"chat1"}}

In [16]:
 response=with_message_history.invoke(

    
    [
        HumanMessage(content="Hi , my name is Siddhesh and i am a data analyst")
      
    ],
    config=config
)

In [18]:
response.content

"It seems like you've told me your profession twice already. No worries, I'm happy to chat with you about anything related to data analysis or your work.\n\nIf you'd like, we could discuss some popular topics in data analysis, such as:\n\n1. Data visualization tools (e.g., Tableau, Power BI, D3.js)\n2. Machine learning algorithms and their applications\n3. Data warehousing and ETL (Extract, Transform, Load) processes\n4. Statistical analysis and hypothesis testing\n5. Data storytelling and communication techniques\n\nLet me know if any of these topics interest you, or if you'd rather talk about something else."

In [19]:
with_message_history.invoke(

    
    [
        HumanMessage(content="what is my name?")
      
    ],
    config=config
)

AIMessage(content='Your name is Siddhesh.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 427, 'total_tokens': 435, 'completion_time': 0.003808798, 'prompt_time': 0.025630664, 'queue_time': 0.054890776, 'total_time': 0.029439462}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--47514e86-d848-4b09-8f3d-af05b386c347-0', usage_metadata={'input_tokens': 427, 'output_tokens': 8, 'total_tokens': 435})

In [20]:
# create another session id call as chat2 and new configuratin
config1={"configurable":{"session_id":"chat2"}}


new_response=with_message_history.invoke(

    
    [
        HumanMessage(content=" my name is sagar and i am a student")
      
    ],
    config=config1
)
new_response

AIMessage(content='Nice to meet you, Sagar. What subject are you studying, and what level are you in (high school, undergraduate, or graduate)?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 45, 'total_tokens': 75, 'completion_time': 0.0627752, 'prompt_time': 0.002653312, 'queue_time': 0.049160927, 'total_time': 0.065428512}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e7f5db2c-ec44-4231-8857-3d19a673a4f1-0', usage_metadata={'input_tokens': 45, 'output_tokens': 30, 'total_tokens': 75})

In [22]:
new_response=with_message_history.invoke(

    
    [
        HumanMessage(content=" who am i?")
      
    ],
    config=config1
)
new_response

AIMessage(content="You are Sagar, a student. I'm glad to be having a conversation with you. What would you like to talk about?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 130, 'total_tokens': 158, 'completion_time': 0.050399646, 'prompt_time': 0.007067644, 'queue_time': 0.048576836, 'total_time': 0.05746729}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--67940648-f983-4ce0-a9c2-cc7fe681ee02-0', usage_metadata={'input_tokens': 130, 'output_tokens': 28, 'total_tokens': 158})

## Prompt Templates

Prompts templates help to turn raw user information into as format thtat the LLm can work with. In this case the raw user input is just a message ,which we are passing to the LLM .
lets now make that a bit more complicated. first,lets add in a system message with some custom instructions(but still taking messagees as input). Next we will add in more input besides just the messsages.

In [24]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant .Answer all question to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")

    ]
)
chain=prompt|model


In [25]:
chain.invoke({"messages":[HumanMessage(content="hi my name is siddhesh")]})

AIMessage(content="Nice to meet you, Siddhesh! I'm an assistant here to help with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 59, 'total_tokens': 95, 'completion_time': 0.051854808, 'prompt_time': 0.003150316, 'queue_time': 0.049513983, 'total_time': 0.055005124}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--7fe37df9-fe76-4c9f-b0ae-3970fea495dc-0', usage_metadata={'input_tokens': 59, 'output_tokens': 36, 'total_tokens': 95})

In [26]:
new_with_message_history=RunnableWithMessageHistory(chain,get_session_history=get_session_history)

In [28]:
config2={'configurable':{"session_id":"chat3"}}
response_3=new_with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config2
)
response_3

AIMessage(content="I don't have any information about your name. I'm a helpful assistant, and our conversation just started. I don't have the ability to retain or access any information about individual users, including their names. Each time you interact with me, it's a new conversation. If you'd like to share your name with me, I'm happy to chat with you!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 115, 'total_tokens': 190, 'completion_time': 0.097140045, 'prompt_time': 0.006470984, 'queue_time': 0.050132575, 'total_time': 0.103611029}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--cc68c5ce-dfcf-422f-bb13-316ca50b116c-0', usage_metadata={'input_tokens': 115, 'output_tokens': 75, 'total_tokens': 190})

In [33]:
# add more complexicity

prompt1=ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant .Answer all question to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages")

    ]
)
chain1=prompt1|model

In [34]:
responce4=chain1.invoke({"messages":[HumanMessage(content="hi , my name is siddhesh")],"language":"hindi"})
responce4.content

'नमस्ते सिद्धेश जी, मैं आपकी सहायता के लिए यहाँ हूँ। आपके प्रश्न का उत्तर देने की कोशिश करूँगा। क्या आप कुछ पूछना चाहते हैं, या कोई समस्या का समाधान चाहते हैं?'

lets noe wrap this more complidated chain n a messge Hstory class.  This time,because there are multiple keys in the imput we need to specify the correct key to use to save the chat history.

In [35]:
new_with_message_history_1=RunnableWithMessageHistory(
    chain1,
    get_session_history=get_session_history,
    input_messages_key="messages"
)

In [38]:
config3={"configurable":{"session_id":"chat5"}}



response4=new_with_message_history_1.invoke(
    {"messages":[HumanMessage(content="hi, i am shree")],"language":"hindi"},
    config3
    )

response4.content

'नमस्ते श्री जी, मैं आपकी मदद करने के लिए यहाँ हूँ। क्या आपके पास कोई प्रश्न है जिस पर मैं आपकी सहायता कर सकता हूँ?'

In [39]:
response4=new_with_message_history_1.invoke(
    {"messages":[HumanMessage(content="what is my name")],"language":"hindi"},
    config3
    )

response4.content

'तुम्हारा नाम श्री है।'

## Mangage the conversation History

x one important concept to understand when building chatboat is  how to manage conversation history .if left unmanaged the list of messages will grow unbounded and potentially overflow the context window of the LLM, therefore m it is  important to add a step that limits the size of the messages you are passing in,

Trim_message=it is helper to resuce how many messages we are sending to the model. The trimmer allows us to specigy how many tokens we want to keep alone with other parametes like if we want ot always keep the system messages and wheteher to allow partial messsages.

In [44]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

# sample conversation

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

# use this messages to tream
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [47]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
|prompt1
|model
)

responce=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what is ice cream do i like")],
    "language":"hindi"
    }
)
responce.content

'आइस क्रीम बहुत स्वादिष्ट होती है! आपको यह पसंद है क्या?'

In [48]:
responce=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what math problem did i adk")],
    "language":"hindi"
    }
)
responce.content

'आप ने 2 + 2 का सवाल पूछा था।'

In [49]:
# lets wrap this in the messge histry
with_message_history_2=RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="messages",

)
config4={"configurable":{"session_id":"chat6"}}


In [50]:
response5=with_message_history_2.invoke(
    {"messages":[HumanMessage(content="what math problem did i ask")],
     "language":"hindi"
     },
     config=config4
)

responce4.content

'नमस्ते सिद्धेश जी, मैं आपकी सहायता के लिए यहाँ हूँ। आपके प्रश्न का उत्तर देने की कोशिश करूँगा। क्या आप कुछ पूछना चाहते हैं, या कोई समस्या का समाधान चाहते हैं?'